# Hello Julia

とりあえずJuliaを触ってみる。触ってみるだけ。

In [1]:
s = "Hello Julia!"
println(s);

Hello Julia!


# 基本構文

## 変数

In [2]:
x = 1
y = 2.0
z = x + y

3.0

こういう書き方もできる。変数の前に数値を置くと暗黙的に乗算と解釈される。

In [3]:
2x + 1

3

## 定数

In [4]:
const a = 1

1

In [5]:
a = 2

2

In [6]:
pi

π = 3.1415926535897...

In [7]:
pi = 3

LoadError: cannot assign a value to variable MathConstants.pi from module Main

In [8]:
pi

π = 3.1415926535897...

## 文字列の補間

In [9]:
"1 + 2 = $(1 + 2)"

"1 + 2 = 3"

# 制御構文

## 条件評価

In [10]:
x = 1
y = 2

if x < y
    println("x < y")
elseif x > y
    println("x > y")
else
    println("x = y")
end

x < y


三項演算子

In [11]:
x < y ? "x<y" : "x>=y"

"x<y"

## 短絡評価

複数の条件を評価する。
最初に書かれた条件の結果によって二つ目の評価を飛ばすことがある。

a && b の場合に、aの結果がFalseならこの結果は必ずFalseになるのでbの評価をしない。

In [12]:
n = 1
n < 0 && error("n must be positive")

false

## ループ

In [13]:
i = 1
while i <= 3
    println("i = $(i)")
    i += 1
end

i = 1
i = 2
i = 3


In [14]:
for j = 1:3
    println("j = $(j)")
end

j = 1
j = 2
j = 3


forやwhileなどのブロック内で使われる変数はブロック内のローカル変数となる。
なので、ブロック外で使用するとエラーが出る。

In [15]:
j

LoadError: UndefVarError: j not defined

## 関数

関数は基本は`function`~`end`で囲む。returnを明示しない場合、最後に評価された値が戻り値となる。

型は指定してもしなくてもOK。

In [16]:
function add(x::Int, y::Int)::Int
    return x + y
end

add (generic function with 1 method)

In [17]:
add(1, 3)

4

1行で関数を書ける場合

In [18]:
add(x::Int, y::Int)::Float64 = x + y

add (generic function with 1 method)

In [19]:
add(3, 4)

7.0

戻り値が複数の場合

In [20]:
function add_diff(x, y)
    return (x+y, x-y)
end

# add_diff(x, y) = (x+y, x-y);

In [21]:
add_diff(4, 3)

(7, 1)

可変長引数

In [22]:
function f(x...)
    sum = 0
    for i = 1:length(x)
        sum += x[i]
    end
    return sum
end

f (generic function with 1 method)

In [23]:
f(1)

1

In [24]:
f(1,2,3,4,5)

15

オプショナル引数

引数名を指定できないけど、デフォルト引数を指定できる。

In [25]:
function f_op(x, y=1)
    return x + y
end

f_op (generic function with 2 methods)

In [26]:
f_op(1)

2

In [27]:
f_op(1, 0)

1

In [28]:
f_op(1, y=0)

LoadError: MethodError: no method matching f_op(::Int64; y=0)
[0mClosest candidates are:
[0m  f_op(::Any) at In[25]:1[91m got unsupported keyword argument "y"[39m
[0m  f_op(::Any, [91m::Any[39m) at In[25]:1[91m got unsupported keyword argument "y"[39m

キーワード引数

引数名を指定できる。

In [29]:
function f_kw(x; y=1)
    return x + y
end

f_kw (generic function with 1 method)

In [30]:
f_kw(1)

2

In [31]:
f_kw(1, y=0)

1

In [32]:
f_kw(1, 0)

LoadError: MethodError: no method matching f_kw(::Int64, ::Int64)
[0mClosest candidates are:
[0m  f_kw(::Any; y) at In[29]:1

# 型

## 複合型

structで宣言する。複数の名前付きフィールドをまとめる。

In [33]:
struct Point
    x::Int
    y::Int
end

In [34]:
function distance(p::Point)
    sqrt(p.x^2 + p.y^2)
end

distance (generic function with 1 method)

In [35]:
p = Point(2, 3)
distance(p)

3.605551275463989

In [36]:
p.x

2

In [37]:
Point(x) = Point(x, 0)

Point

In [38]:
p = Point(1)
p

Point(1, 0)

In [39]:
Point(1, 5)

Point(1, 5)

structで宣言したフィールドはオブジェクトの作成の後からは変更できなくなる

In [40]:
p.x = 2

LoadError: setfield! immutable struct of type Point cannot be changed

mutableなstruct宣言をすると、後からフィールドの値を変更できる

## パラメトリック型

In [41]:
mutable struct Point_p{T}
    x::T
    y::T
end;

In [42]:
p = Point_p(2, 3) # TはIntと解釈される

Point_p{Int64}(2, 3)

In [43]:
p = Point_p(2.1, 3.) # TはFloatと解釈される

Point_p{Float64}(2.1, 3.0)

In [44]:
function distance(p::Point_p{T}) where T
    sqrt(p.x^2 + p.y^2)
end

distance (generic function with 2 methods)

In [45]:
distance(p)

3.661966684720111

型によって動作を変える場合は、多重ディスパッチを利用する

In [46]:
function distance(p::Point_p{Int})
    println("int")
    sqrt(p.x^2 + p.y^2)
end;

function distance(p::Point_p{Float64})
    println("float64")
    sqrt(p.x^2 + p.y^2)
end;

In [47]:
p = Point_p(2, 3)
distance(p)

int


3.605551275463989

In [48]:
p = Point_p(2.5, 3.5)
distance(p)

float64


4.301162633521313